In [5]:
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pyro
import gpytorch
import torch
import math
import msc
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
max_time = 45

times = torch.linspace(0, max_time, 1024)


In [7]:
true_intensity_function = cycles_intensity_function()

NameError: name 'cycles_intensity_function' is not defined

In [ ]:
true_intensity_function.plot_graph(times)

In [ ]:
num_samples = int(pyro.distributions.Poisson(true_intensity_function(times).mean() * max_time).sample().item())
print(f"{num_samples=}")
def log_prob_accept(val):
    intensities = true_intensity_function(val)
    res = torch.log(intensities / (true_intensity_function(times).mean() * max_time))
    return res

arrival_times = pyro.distributions.Rejector(
    propose=pyro.distributions.Uniform(times.min(), times.max()),
    log_prob_accept=log_prob_accept,
    log_scale=0.
)(torch.Size([num_samples]))

In [ ]:
plt.plot(times, true_intensity_function(times), label="true density")
binwidth = 0.5
plt.hist(arrival_times.tolist(), bins=np.arange(min(times), max(times), binwidth), label="simulated data density")
plt.legend();